## IMPORT LIB

## PREDIKSI

In [ ]:
# prediksi user-based

def prediksiCF(RatingMatriks, similarityFunction, mean, meanCen, user="user-2", item="item-2", k=2, jenis="userBased"):

    if jenis == "userBased":
        # mendapatkan item yang diberi rating oleh pengguna
        ratingMatriks = RatingMatriks.loc[:, item].to_numpy()
        # print(ratingMatriks)
        # mendapatan mean centered
        meanCentered = meanCen.loc[:, item].to_numpy()
        print("mean centered",meanCentered)
        # mendapatkan similarity
        similarity = similarityFunction.loc[user, :].to_numpy()
        # print(similarity)

    if jenis == "itemBased":
        # mendapatkan item yang diberi rating oleh pengguna
        ratingMatriks = RatingMatriks.loc[user, :].to_numpy()
        # print(ratingMatriks)
        # mendapatan mean centered
        meanCentered = meanCen.loc[user, :].to_numpy()
        # print(meanCentered)
        # mendapatkan similarity
        similarity = similarityFunction.loc[:, item].to_numpy()
        # print(similarity)

    # cek index data user/item yang tidak sama dengan 0
    cek_Index = np.where(ratingMatriks != 0)

    # filter user/item rating, mean centered, similarity, sesuai index data yang tidak bernilai 0
    nilai_meanCentered = np.array(meanCentered)[cek_Index]
    # print("cek nilai meanCenterd", nilai_meanCentered)
    nilai_similarity = similarity[cek_Index]
    print("cek nilai similarity", nilai_similarity)
    # tetangga terdekat
    indexTetanggaSim = TopKTetangga(nilai_similarity, k)
    print("cek tetangga ", indexTetanggaSim)

    # pembilang 
    pembilang = np.sum(nilai_meanCentered[indexTetanggaSim] * nilai_similarity[indexTetanggaSim])
    # print("pembilang" ,pembilang)
    # penyebut
    penyebut = np.abs(nilai_similarity[indexTetanggaSim]).sum()
    # print("pennyebuat" ,penyebut)

    # cek apakah penyebut tidak sama dengan 0
    if penyebut != 0:
        if jenis == "userBased":
            # prediksi rating
            rumusPrediksi = mean.loc[user] + (pembilang / penyebut)
        elif jenis == "itemBased":
            # rumusPrediksi rating
            rumusPrediksi = mean.loc[item] + (pembilang / penyebut)
    else:
        if jenis == "userBased":
            rumusPrediksi = mean.loc[user] + 0
        elif jenis == "itemBased":
            rumusPrediksi = mean.loc[item] + 0

    return [item, float(rumusPrediksi)] 
    

## TOPN 

In [ ]:
def getTopNRekomendasiV2(prediksiHybrid, ratingMatrix, user_index, N=5):
    """
    Mengembalikan Top-N rekomendasi berupa ID item saja (tanpa skor),
    hanya untuk item yang belum diberi rating, berdasarkan prediksi hybrid.
    """
    pred_user = prediksiHybrid.iloc[user_index].values
    rating_user = ratingMatrix.iloc[user_index].values

    # Item yang belum dirating (nilai 0 atau NaN)
    if np.issubdtype(rating_user.dtype, np.number):
        belum_dirating_idx = np.where(rating_user == 0)[0]
    else:
        belum_dirating_idx = np.where(pd.isna(rating_user))[0]

    # Prediksi hanya untuk item yang belum dirating
    pred_belum_dirating = pred_user[belum_dirating_idx]

    # Urutkan dari skor tertinggi
    top_indices = pred_belum_dirating.argsort()[::-1][:N]

    # Ambil ID item-nya (pakai to_numpy() supaya aman)
    top_item_ids = prediksiHybrid.columns.to_numpy()[belum_dirating_idx][top_indices]

    return list(top_item_ids)


In [46]:
import pandas as pd 
import numpy as np
import math as math

In [ ]:
for i in range(1,6):
    basedata, testdata = load_Dataset(call_base, 1)

    fileName = f'testDataExel_{i}.xlsx'
    testdata.to_excel(fileName, index=False)
    print(f"Base data for fold {i} saved to {fileName}")

In [47]:
def readData():
    #rename columns untuk data frame
    rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
    # read data from u.data file
    readData = pd.read_csv("ml-100k/u1.base", sep="\t", header=None, names=rnmColData)
    # drop timestamp column
    readData = readData.drop('timestamp', axis=1)
    # return readData
    return readData

In [48]:
cal_ReadData = readData()
cal_ReadData

,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
79995,943,1067,2
79996,943,1074,4
79997,943,1188,3
79998,943,1228,3


In [49]:
# filter 5 teratas

df_Filter = cal_ReadData[cal_ReadData['user_id']==1]
# df_FilterSort = df_Filter.sort_values(by='rating', ascending=False)
top5 = df_Filter.head(5)

In [50]:
print(top5)

   user_id  item_id  rating
0        1        1       5
1        1        2       3
2        1        3       4
3        1        4       3
4        1        5       3


In [51]:
#  datasetInput = ratingData.pivot(index = 'user_id', columns = 'item_id', values='rating')
#     # fill NaN with 0
#     insertDataNew = datasetInput.fillna(0)

In [52]:
def ConvertRatingMatriks(ratingData):
    # membuat container untuk rating matriks dengan ukuran user x item
    matriks_rating = pd.DataFrame(np.zeros((943, 1682)), columns = list(range(1, 1683)), index = list(range(1, 944)))
    # merubah data frame ke dalam bentuk matriks rating pivot
    convertMatriksRating = ratingData.pivot_table(index = 'user_id', columns = 'item_id', values='rating')
    # mengisi matriks rating NaN dengan 0
    matriks_rating = convertMatriksRating.fillna(0)
    # updating matriks rating dengan matriks rating yang sudah di pivot
    matriks_rating.update(convertMatriksRating)
    # mengembalikan matriks rating
    return matriks_rating

In [53]:
rating_matriks = ConvertRatingMatriks(cal_ReadData)
# matriks rating
rating_matriks

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TOY DATA

In [54]:
toyDataMatriks = [
    [3,	0,	0,	0,	0,	0,	2],
    [1,	2,	4,	2,	2,	0,	0],
    [0,	4,	1,	0,	0,	2,	4],
    [0,	0,	0,	0,	3,	2,	4]
]

In [55]:
# toydata to dataframe

dfToyData = pd.DataFrame(toyDataMatriks, columns=['item-1', 'item-2', 'item-3', 'item-4','item-5','item-6', 'item-7'], index=['user-1', 'user-2', 'user-3', 'user-4'])
dfToyData


,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3,0,0,0,0,0,2
user-2,1,2,4,2,2,0,0
user-3,0,4,1,0,0,2,4
user-4,0,0,0,0,3,2,4


## MEAN

In [56]:
def meanJaccard(rating):
    pembilang = rating.sum(axis=1)
    penyebut = np.count_nonzero(rating, axis=1)
    # menghindari pembagian dengan 0
    rumusMean = pembilang / penyebut
    calCulatedMean  = np.where(penyebut != 0, rumusMean, 0)
    # menghitung mean
    meanRatingUser = pd.DataFrame(calCulatedMean, index=rating.index, columns=['meanRating'])
    return meanRatingUser

In [57]:
cal_MeanUser = meanJaccard(dfToyData)
cal_MeanUser

,meanRating
user-1,2.50
user-2,2.20
user-3,2.75
user-4,3.00


In [58]:
cal_MeanItem = meanJaccard(dfToyData.T)
cal_MeanItem

,meanRating
item-1,2.000000
item-2,3.000000
item-3,2.500000
item-4,2.000000
item-5,2.500000
item-6,2.000000
item-7,3.333333


In [59]:
# # tranpose

# cal_MeanItemTranpose = cal_MeanItem.T
# cal_MeanItemTranpose

## MEAN CENTERED

In [60]:
def meanCenterd(rating, meanRating):
    npRatingMatriks = np.array(rating)
    npMeanRatingUser = np.array(meanRating)
    # !0
    rumusMeanCenterd = npRatingMatriks - npMeanRatingUser
    meanCenteredUser = np.where(npRatingMatriks!=0, rumusMeanCenterd,0)
    dfMeanCenteredUser = pd.DataFrame(meanCenteredUser, index=rating.index, columns=rating.columns)

    return dfMeanCenteredUser

In [61]:
cal_MeanCenteredUser = meanCenterd(dfToyData, cal_MeanUser)
cal_MeanCenteredUser

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,0.5,0.00,0.00,0.0,0.0,0.00,-0.50
user-2,-1.2,-0.20,1.80,-0.2,-0.2,0.00,0.00
user-3,0.0,1.25,-1.75,0.0,0.0,-0.75,1.25
user-4,0.0,0.00,0.00,0.0,0.0,-1.00,1.00


In [62]:
cal_MeanCenteredItem = meanCenterd(dfToyData.T, cal_MeanItem)
cal_MeanCenteredItem


,user-1,user-2,user-3,user-4
item-1,1.000000,-1.0,0.000000,0.000000
item-2,0.000000,-1.0,1.000000,0.000000
item-3,0.000000,1.5,-1.500000,0.000000
item-4,0.000000,0.0,0.000000,0.000000
item-5,0.000000,-0.5,0.000000,0.500000
item-6,0.000000,0.0,0.000000,0.000000
item-7,-1.333333,0.0,0.666667,0.666667


In [63]:
cal_MeanCenteredItemTranpose = cal_MeanCenteredItem.T
cal_MeanCenteredItemTranpose

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,1.0,0.0,0.0,0.0,0.0,0.0,-1.333333
user-2,-1.0,-1.0,1.5,0.0,-0.5,0.0,0.000000
user-3,0.0,1.0,-1.5,0.0,0.0,0.0,0.666667
user-4,0.0,0.0,0.0,0.0,0.5,0.0,0.666667


## USER-BASED

In [64]:
def simJaccardUser(rating, penggunaU, penggunaV):
    # mendapatkan item yang diberi rating oleh penggunaU dan penggunaV
    itemRatedPenggunaU = set(np.where(rating.loc[penggunaU, :] != 0)[0]) 
    # print("Item yang diberi rating oleh pengguna U:", itemRatedPenggunaU)
    # print(penggunaU, "penggunaU")
    itemRatedPenggunaV = set(np.where(rating.loc[penggunaV, :] != 0)[0]) 

    # cekRateUserU = np.where(rating.loc[penggunaU, :] != 0)
    # print(cekRateUserU, "rateUserU")
    # finalRate = cekRateUserU[0]
    # print("final", finalRate)
    # displayDataItemRated = set(rating.columns[list(itemRatedPenggunaU)])
    # print("Item yang diberi rating oleh pengguna U:", displayDataItemRated)
    # disSp = rating.columns[list(cekRateUserU)]
    # print("Item yang diberi rating oleh pengguna U:", disSp)
    # Menghitung irisan (intersection) dari item yang diberi rating oleh kedua pengguna
    intersection = len(itemRatedPenggunaU.intersection(itemRatedPenggunaV))
    # print(intersection)
    # Menghitung gabungan (union) dari item yang diberi rating oleh kedua pengguna
    union = len(itemRatedPenggunaU.union(itemRatedPenggunaV))
    # print(union)
    rumusJaccard = intersection/union

    # Menghitung similarity Jaccard
    # cari apakah boleh
    if union != 0:
        return rumusJaccard
    else:
        return 0
    

In [65]:
callSimJaccardUser = simJaccardUser(dfToyData, 'user-1', 'user-3')
print(f"Jaccard Similarity between user-1 and user-2 is {callSimJaccardUser}")

Jaccard Similarity between user-1 and user-2 is 0.2


In [66]:
def similarityJaccardPengguna(matriksRating):
    # Mendapatkan jumlah pengguna baris
    jumlahPengguna = matriksRating.shape[0]
    # Membuat matriks similarity kosong
    matriksSimilarity = np.zeros((jumlahPengguna, jumlahPengguna))
    # Menghitung similarity Jaccard untuk setiap pasangan pengguna
    for i in range(jumlahPengguna):
        for j in range(i, jumlahPengguna):
            if i == j:
                matriksSimilarity[i][j] = 1  # Jika pengguna yang dibandingkan sama, similarity adalah 1
                continue
            rumusSimJaccardUser = simJaccardUser(matriksRating, i, j)
            # rumusSimJaccardUser = simJaccardUser(matriksRating, matriksRating.index[i], matriksRating.index[j])
            matriksSimilarity[i][j] = rumusSimJaccardUser
            matriksSimilarity[j][i] = rumusSimJaccardUser
    # Mengembalikan hasil dalam bentuk DataFrame
    dfMatriksSimilarity = pd.DataFrame(matriksSimilarity, index=matriksRating.index, columns=matriksRating.index)
    
    return dfMatriksSimilarity

In [67]:
# cal_Sim = similarityJaccardPengguna(dfToyData)
# cal_Sim

In [68]:
def simJaccardDuaUser(rating, penggunaU, penggunaV):
    # Dapatkan item yang dirating (tidak sama dengan 0)
    itemRatedPenggunaU = set(rating.columns[rating.iloc[penggunaU, : ] != 0])
    itemRatedPenggunaV = set(rating.columns[rating.iloc[penggunaV, : ] != 0])

    # Hitung Jaccard similarity
    intersection = len(itemRatedPenggunaU.intersection(itemRatedPenggunaV))
    union = len(itemRatedPenggunaU.union(itemRatedPenggunaV))

    if union != 0:
        return intersection / union
    else:
        return 0


In [69]:
cal_SimDuaUser = simJaccardDuaUser(dfToyData, 0, 2)
print(f"Jaccard Similarity between user-1 and user-3 is {cal_SimDuaUser}")

Jaccard Similarity between user-1 and user-3 is 0.2


In [70]:
def simiALLUser(MatriksRating):
    #jumlah user baris [0]
    jumlahPengguna = MatriksRating.shape[0]
    matriksSimilarity = np.zeros((jumlahPengguna, jumlahPengguna))
    np.fill_diagonal(matriksSimilarity, 1)

    for i in range (jumlahPengguna):
        for j in range(i+1, jumlahPengguna):
            rumusSimJaccardUser = simJaccardDuaUser(MatriksRating, i, j)
            matriksSimilarity[i][j] = rumusSimJaccardUser
            matriksSimilarity[j][i] = rumusSimJaccardUser
    
    # Mengembalikan hasil dalam bentuk DataFrame
    dfMatriksSimilarity = pd.DataFrame(matriksSimilarity, index=MatriksRating.index, columns=MatriksRating.index)

    return dfMatriksSimilarity


In [71]:
cal_simAll = simiALLUser(dfToyData)
cal_simAll

,user-1,user-2,user-3,user-4
user-1,1.000000,0.166667,0.200000,0.250000
user-2,0.166667,1.000000,0.285714,0.142857
user-3,0.200000,0.285714,1.000000,0.400000
user-4,0.250000,0.142857,0.400000,1.000000


## ITEM-BASED

In [72]:
# def simJaccardItem(RatingMatriks, itemI, itemJ) :
#     # mendapatkan pengguna yang memberi rating pada itemI dan itemJ
#     userRatedItemI = set(np.where(RatingMatriks.loc[:, itemI] != 0)[0])  # Indeks pengguna yang memberi rating pada itemI
#     userRatedItemJ = set(np.where(RatingMatriks.loc[:, itemJ] != 0)[0])  # Indeks pengguna yang memberi rating pada itemJ

#     # Menghitung irisan (intersection) dari pengguna yang memberi rating pada kedua item
#     intersection = len(userRatedItemI.intersection(userRatedItemJ))
#     # Menghitung gabungan (union) dari pengguna yang memberi rating pada kedua item
#     union = len(userRatedItemI.union(userRatedItemJ))
#     # rumus jaccard item
#     if union != 0 :
#         rumusJac = intersection/union
#     else :
#         rumusJac = 0
#     return rumusJac

In [73]:
# cal_simJaccardItem = simJaccardItem(dfToyData, 'item-2', 'item-3')
# print(f"Jaccard Similarity between item-1 and item-2 is {cal_simJaccardItem}")

## ITEM BASED

In [74]:
def simJaccardDuaItem(rating, itemI, itemJ):
    # mendapatkan user yang sudah memberikan rating pada item I
    itemRatedItemI = set(rating.index[rating.iloc[:, itemI] != 0])
    # user yang sudah memberikan raitng pada item J
    itemRatedItemJ = set(rating.index[rating.iloc[:, itemJ] != 0])


    # Hitung Jaccard similarity
    intersection = len(itemRatedItemI.intersection(itemRatedItemJ))
    union = len(itemRatedItemI.union(itemRatedItemJ))

    rumusJaccardItem = intersection/union
    if intersection != 0:
        return rumusJaccardItem
    else:
        return 0


In [75]:
simJaccardDuaItemW = simJaccardDuaItem(dfToyData, 0, 1)
print(f"Jaccard Similarity between item-1 and item-2 is {simJaccardDuaItemW}")

Jaccard Similarity between item-1 and item-2 is 0.3333333333333333


In [76]:
def simiALLItem(MatriksRating):
    #jumlah item kolom [1]
    jumlahItem = MatriksRating.shape[1]
    matriksSimilarity = np.zeros((jumlahItem, jumlahItem))
    np.fill_diagonal(matriksSimilarity, 1)

    for i in range (jumlahItem):
        for j in range(i+1, jumlahItem):
            rumusSimJaccardItem = simJaccardDuaItem(MatriksRating, i, j)
            matriksSimilarity[i][j] = rumusSimJaccardItem
            matriksSimilarity[j][i] = rumusSimJaccardItem
    
    # Mengembalikan hasil dalam bentuk DataFrame
    dfMatriksSimilarity = pd.DataFrame(matriksSimilarity, index=MatriksRating.columns, columns=MatriksRating.columns)

    return dfMatriksSimilarity


In [77]:
cal_Item = simiALLItem(dfToyData)
cal_Item

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
item-1,1.000000,0.333333,0.333333,0.5,0.333333,0.000000,0.250000
item-2,0.333333,1.000000,1.000000,0.5,0.333333,0.333333,0.250000
item-3,0.333333,1.000000,1.000000,0.5,0.333333,0.333333,0.250000
item-4,0.500000,0.500000,0.500000,1.0,0.500000,0.000000,0.000000
item-5,0.333333,0.333333,0.333333,0.5,1.000000,0.333333,0.250000
item-6,0.000000,0.333333,0.333333,0.0,0.333333,1.000000,0.666667
item-7,0.250000,0.250000,0.250000,0.0,0.250000,0.666667,1.000000


In [78]:
# def similarityJaccardItem(RatingMatriks):
#     # Mendapatkan jumlah item
#     jumlahItem = RatingMatriks.shape[1]

#     # Membuat matriks similarity kosong
#     matriksSimilarity = np.zeros((jumlahItem, jumlahItem))

#     # Menghitung similarity Jaccard untuk setiap pasangan item
#     for i in range(jumlahItem):
#         for j in range(i, jumlahItem):
#             if i == j:
#                 matriksSimilarity[i][j] = 1  # Jika item yang dibandingkan sama, similarity adalah 1
#                 continue
#             rumusSimJaccardItem = simJaccardItem(RatingMatriks, RatingMatriks.columns[i], RatingMatriks.columns[j])
#             matriksSimilarity[i][j] =  rumusSimJaccardItem
#             matriksSimilarity[j][i] = rumusSimJaccardItem

#     # Mengembalikan hasil dalam bentuk DataFrame
#     dfMatriksSimilarity = pd.DataFrame(matriksSimilarity, index=RatingMatriks.columns, columns=RatingMatriks.columns)
    
#     return dfMatriksSimilarity

In [79]:
# cal_similarityJaccardItem = similarityJaccardItem(dfToyData)
# cal_similarityJaccardItem

In [ ]:
def PrediksiRating(RatingMatriks, similarityFunction, mean, meanCen, user=2, item=2, k=2, jenis="user-based"):
    if jenis == "user-based":
        rating_Matriks = RatingMatriks.loc[:, item].to_numpy()
        meanCenterd = meanCen.loc[:, item].to_numpy()
        sim = similarityFunction.loc[user, :].to_numpy()
    elif jenis == "item-based":
        rating_Matriks = RatingMatriks.loc[user, :].to_numpy()
        meanCenterd = meanCen.loc[user, :].to_numpy()
        sim = similarityFunction.loc[:, item].to_numpy()

    TetanggaIndex = TetanggaK2(user, item, RatingMatriks, similarityFunction, k, jenis=jenis)

    # Konversi label (user/item) ke posisi indeks array numpy
    if jenis == "user-based":
        TetanggaPosisi = [RatingMatriks.index.get_loc(i) for i in TetanggaIndex]
    elif jenis == "item-based":
        TetanggaPosisi = [RatingMatriks.columns.get_loc(i) for i in TetanggaIndex]

    nilaiMeanCentered = meanCenterd[TetanggaPosisi]
    nilaiSim = sim[TetanggaPosisi]

    pembilang = np.sum(nilaiMeanCentered * nilaiSim)
    penyebut = np.abs(nilaiSim).sum()

    if penyebut != 0:
        if jenis == "user-based":
            prediksi = mean.loc[user] + (pembilang / penyebut)
        elif jenis == "item-based":
            prediksi = mean.loc[item] + (pembilang / penyebut)
    else:
        if jenis == "user-based":
            prediksi = mean.loc[user]
        elif jenis == "item-based":
            prediksi = mean.loc[item]

    return [item, float(prediksi)]


In [80]:
def similarityAllUserItem(rating, jenis="user"):
    if jenis == "user" : 
        jumlahUser = rating.shape[0]
        matriksSimilarity = np.zeros((jumlahUser, jumlahUser))
        np.fill_diagonal(matriksSimilarity, 1)
        for i in range (jumlahUser):
            for j in range(i+1, jumlahUser):
                rumusSimJaccardUser = simJaccardDuaUser(rating, i, j)
                matriksSimilarity[i][j] = rumusSimJaccardUser
                matriksSimilarity[j][i] = rumusSimJaccardUser
        simMatriksDF = pd.DataFrame(matriksSimilarity, index=rating.index, columns=rating.index)
    elif jenis == "item" : 
        jumlahItem = rating.shape[1]
        matriksSimilarity = np.zeros((jumlahItem, jumlahItem))
        np.fill_diagonal(matriksSimilarity, 1)
        for i in range (jumlahItem):
            for j in range(i+1, jumlahItem):
                rumusSimJaccardItem = simJaccardDuaItem(rating, i, j)
                matriksSimilarity[i][j] = rumusSimJaccardItem
                matriksSimilarity[j][i] = rumusSimJaccardItem
        simMatriksDF = pd.DataFrame(matriksSimilarity, index=rating.columns, columns=rating.columns)
    return simMatriksDF

In [81]:
cal_SimUserJaccard = similarityAllUserItem(dfToyData, "user")
cal_SimUserJaccard

,user-1,user-2,user-3,user-4
user-1,1.000000,0.166667,0.200000,0.250000
user-2,0.166667,1.000000,0.285714,0.142857
user-3,0.200000,0.285714,1.000000,0.400000
user-4,0.250000,0.142857,0.400000,1.000000


In [82]:
cal_SimItemJaccard = similarityAllUserItem(dfToyData, "item")
cal_SimItemJaccard

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
item-1,1.000000,0.333333,0.333333,0.5,0.333333,0.000000,0.250000
item-2,0.333333,1.000000,1.000000,0.5,0.333333,0.333333,0.250000
item-3,0.333333,1.000000,1.000000,0.5,0.333333,0.333333,0.250000
item-4,0.500000,0.500000,0.500000,1.0,0.500000,0.000000,0.000000
item-5,0.333333,0.333333,0.333333,0.5,1.000000,0.333333,0.250000
item-6,0.000000,0.333333,0.333333,0.0,0.333333,1.000000,0.666667
item-7,0.250000,0.250000,0.250000,0.0,0.250000,0.666667,1.000000


## PREDIKSI USER-BASED

### TOP-K

In [ ]:
def TetanggaK3(target_user, target_item, matriks_rating, similarity, k, jenis="user-based"):
    # jika jenis user-based
    if jenis == "user-based":
        # daftar user lain yang sudah memberi rating pada item target
        # memilih user yang tidak sama dengan target user
        # dan hanya memilih user yang telah memberikan rating pada item target, yang lebih dari 0
        # himpunan user yang telah memberikan rating pada item target (ui) index
        itemRated1 = np.where((matriks_rating).iloc[:, target_item] != 0)[0]
        itemFilterRated = itemRated1[itemRated1 != target_user]
        # Ambil nilai similarity terhadap user target yang ada di filter indices, baris target user dan kolom user pada filter, array
        sim_scores = similarity.iloc[target_user, :].to_numpy()
        nilaiSim = sim_scores[itemFilterRated]
        
    # jika jenis item-based
    elif jenis == "item-based":
        # Ambil item-item lain yang pernah diberi rating oleh user target
        # memilih item yang tidak sama dengan target item
        # dan hanya memilih item yang telah memberikan rating pada user target, yang lebih dari 0
        itemRated1 = np.where((matriks_rating).iloc[target_user,:] != 0)[0]
        itemFilterRated = itemRated1[itemRated1 != target_item]
        # Ambil nilai similarity terhadap item target yang ada di filter indices, baris target item dan kolom item pada filter, array
        sim_scores = similarity.iloc[:, target_item].to_numpy()
        nilaiSim = sim_scores[itemFilterRated]
    
    # Mengambil indeks dari k yang paling mirip, yang paling besar, mengambil k indeks pertama
    top_k_idx = np.argsort(-nilaiSim)[:k]
    
    # Mapping ulang sesuai dengan index pada rating matriks
    top_k_items = [itemFilterRated[i] for i in top_k_idx]
    # print(nilaiSim[top_k_idx])
    
    return top_k_items


In [83]:
def TopKTetangga(similarity, k):
    return np.argsort(-similarity)[:k]  # Mengambil k tetangga terdekat berdasarkan sim

In [84]:
cal_Tokk = TopKTetangga(cal_simAll, 4)
cal_Tokk

,user-1,user-2,user-3,user-4
user-1,0,3,2,1
user-2,1,2,0,3
user-3,2,3,1,0
user-4,3,2,0,1


In [85]:
# def TopKTetangga2(similarity, k):
#     result = []
#     for i in range(similarity.shape[0]):
#         sim_row = similarity.iloc[i].copy()
#         sim_row.iloc[i] = -np.inf  # Hindari diri sendiri
#         top_k = np.argsort(-sim_row.values)[:k]  # ambil nilai tertinggi
#         result.append(top_k)
#     return np.array(result)


In [86]:
# # calTetanga2 = TopKTetangga2(cal_similarityJaccardUser, 2)
# calTetanga2

In [87]:
# prediksi user-based

def prediksiCF(RatingMatriks, similarityFunction, mean, meanCen, user="user-2", item="item-2", k=2, jenis="userBased"):

    if jenis == "userBased":
        # mendapatkan item yang diberi rating oleh pengguna
        ratingMatriks = RatingMatriks.loc[:, item].to_numpy()
        # print(ratingMatriks)
        # mendapatan mean centered
        meanCentered = meanCen.loc[:, item].to_numpy()
        # print(meanCentered)
        # mendapatkan similarity
        similarity = similarityFunction.loc[user, :].to_numpy()
        # print(similarity)

    if jenis == "itemBased":
        # mendapatkan item yang diberi rating oleh pengguna
        ratingMatriks = RatingMatriks.loc[user, :].to_numpy()
        # print(ratingMatriks)
        # mendapatan mean centered
        meanCentered = meanCen.loc[user, :].to_numpy()
        # print(meanCentered)
        # mendapatkan similarity
        similarity = similarityFunction.loc[:, item].to_numpy()
        # print(similarity)

    # cek index data user/item yang tidak sama dengan 0
    cek_Index = np.where(ratingMatriks != 0)

    # filter user/item rating, mean centered, similarity, sesuai index data yang tidak bernilai 0
    nilai_meanCentered = np.array(meanCentered)[cek_Index]
    # print("cek nilai meanCenterd", nilai_meanCentered)
    nilai_similarity = similarity[cek_Index]
    # print("cek nilai similarity", nilai_similarity)
    # tetangga terdekat
    indexTetanggaSim = TopKTetangga(nilai_similarity, k)
    # print("cek tetangga ", indexTetanggaSim)

    # pembilang 
    pembilang = np.sum(nilai_meanCentered[indexTetanggaSim] * nilai_similarity[indexTetanggaSim])
    # print("pembilang" ,pembilang)
    # penyebut
    penyebut = np.abs(nilai_similarity[indexTetanggaSim]).sum()
    # print("pennyebuat" ,penyebut)

    # cek apakah penyebut tidak sama dengan 0
    if penyebut != 0:
        if jenis == "userBased":
            # prediksi rating
            rumusPrediksi = mean.loc[user] + (pembilang / penyebut)
        elif jenis == "itemBased":
            # rumusPrediksi rating
            rumusPrediksi = mean.loc[item] + (pembilang / penyebut)
    else:
        if jenis == "userBased":
            rumusPrediksi = mean.loc[user] + 0
        elif jenis == "itemBased":
            rumusPrediksi = mean.loc[item] + 0

    return [item, float(rumusPrediksi)] 
    

In [88]:
cal_PrediksiJaccardUser = prediksiCF(dfToyData, cal_simAll, cal_MeanUser, cal_MeanCenteredUser, user='user-1', item="item-2", k=2, jenis="userBased")
cal_PrediksiJaccardUser

['item-2', 3.090909090909091]

In [89]:
# print("=======================================")
# print(dfToyData)
# print("=======================================")
# print(cal_similarityJaccardItem)
# print("=======================================")
# print(cal_MeanItem)
# print("=======================================")
# print(cal_MeanCenteredItemTranpose)

cal_PrediksiJaccardItem = prediksiCF(dfToyData, cal_Item, cal_MeanItem, cal_MeanCenteredItemTranpose, user="user-1", item="item-2", k=2, jenis="itemBased")
cal_PrediksiJaccardItem

['item-2', 3.0]

### PREDIKSI KESELURUHAN 

In [90]:
def hitungKeseluruhanPrediksi(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="userBased"):
    # Membuat DataFrame kosong untuk menyimpan prediksi
    prediksiMatriks = pd.DataFrame(index=RatingMatriks.index, columns=RatingMatriks.columns)
    
    # Iterasi untuk setiap user dan item dalam matriks rating
    for user in RatingMatriks.index:
        for item in RatingMatriks.columns:
            # Cek apakah rating untuk (user, item) adalah 0 atau tidak ada rating
            if RatingMatriks.loc[user, item] == 0:
                # Jika tidak ada rating, prediksi nilai rating menggunakan Collaborative Filtering
                prediksiMatriks.loc[user, item] = prediksiCF(RatingMatriks, similarityFunction, mean, meanCen, user, item, k, jenis)[1]
            else:
                # Jika sudah ada rating, simpan rating asli dari RatingMatriks
                prediksiMatriks.loc[user, item] = RatingMatriks.loc[user, item]

    return prediksiMatriks


In [91]:
cal_HitungAllPrediksiUser = hitungKeseluruhanPrediksi(dfToyData, cal_simAll, cal_MeanUser, cal_MeanCenteredUser, k=2, jenis="userBased")
cal_HitungAllPrediksiUser

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3,3.090909,2.363636,2.3,2.42,1.611111,2
user-2,1,2,4,2,2,1.366667,2.805263
user-3,2.25,4,1,2.55,2.666667,2,4
user-4,2.881818,3.868421,2.184211,2.8,3,2,4


In [92]:
cal_HitungAllPrediksiItem = hitungKeseluruhanPrediksi(dfToyData, cal_Item, cal_MeanItem, cal_MeanCenteredItemTranpose, k=2, jenis="itemBased")
cal_HitungAllPrediksiItem

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3,3.0,2.5,3.0,2.5,0.666667,2
user-2,1,2,4,2,2,2.25,2.333333
user-3,1.75,4,1,1.75,2.25,2,4
user-4,2.571429,3.25,2.75,2.5,3,2,4


## PREDIKSI ITEM-BASED

## HYBRID

In [93]:
def HybridFiltering(PrediksiUser, PrediksiItem, gamma=0.7):
    # Mengonversi ke numpy array
    prediksiUser = np.array(PrediksiUser)
    prediksiItem = np.array(PrediksiItem)
    # Menghitung prediksi hybrid
    rumusHybrid = gamma * prediksiUser + (1 - gamma) * prediksiItem    
    return rumusHybrid


In [94]:
#menjadi df 
cal_HybridJaccard = pd.DataFrame(HybridFiltering(cal_HitungAllPrediksiUser, cal_HitungAllPrediksiItem, gamma=0.7), index=cal_HitungAllPrediksiUser.index, columns=cal_HitungAllPrediksiUser.columns)
cal_HybridJaccard

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3.0,3.063636,2.404545,2.51,2.444,1.327778,2.0
user-2,1.0,2.0,4.0,2.0,2.0,1.631667,2.663684
user-3,2.1,4.0,1.0,2.31,2.541667,2.0,4.0
user-4,2.788701,3.682895,2.353947,2.71,3.0,2.0,4.0


In [95]:
cal_HybridJac = HybridFiltering(cal_HitungAllPrediksiUser, cal_HitungAllPrediksiItem, gamma=0.7)
cal_HybridJac

array([[3.0, 3.0636363636363635, 2.4045454545454543, 2.51, 2.444,
        1.3277777777777777, 2.0],
       [1.0, 2.0, 4.0, 2.0, 2.0, 1.6316666666666668, 2.663684210526316],
       [2.1, 4.0, 1.0, 2.3099999999999996, 2.5416666666666665, 2.0, 4.0],
       [2.7887012987012985, 3.682894736842105, 2.3539473684210526, 2.71,
        3.0, 2.0, 4.0]], dtype=object)

## TOP N

In [96]:
def TopNRecommendation(rekomendasiHybrid, N=5):
    topN_per_user = np.argsort(-rekomendasiHybrid, axis=1)[:N]+1
    # tolist
    topN_per_user = topN_per_user.tolist()

    return topN_per_user


In [97]:
calTpn = TopNRecommendation(cal_HybridJac)
calTpn

[[2, 1, 4, 5, 3, 7, 6],
 [3, 7, 2, 4, 5, 6, 1],
 [2, 7, 5, 4, 1, 6, 3],
 [7, 2, 5, 1, 4, 3, 6]]

In [98]:
def FullRecommendation(prediksi_matrix):
    ranking_per_user = np.argsort(-prediksi_matrix, axis=1)+1
    return ranking_per_user

In [99]:
cal_TopNRekomendasiFull = FullRecommendation(cal_HybridJac)
cal_TopNRekomendasiFull

array([[2, 1, 4, 5, 3, 7, 6],
       [3, 7, 2, 4, 5, 6, 1],
       [2, 7, 5, 4, 1, 6, 3],
       [7, 2, 5, 1, 4, 3, 6]], dtype=int64)

In [100]:
# jadikan list 1d dan ambil all user

#semua user jadikan list 
convertList1DFull = cal_TopNRekomendasiFull.tolist()
print(convertList1DFull)


[[2, 1, 4, 5, 3, 7, 6], [3, 7, 2, 4, 5, 6, 1], [2, 7, 5, 4, 1, 6, 3], [7, 2, 5, 1, 4, 3, 6]]


## NDCG

In [101]:
# ndcg
groundTruth = [1, 2, 3, 4]
TopNrek = [1, 5, 2, 3, 15]
TopNrek = [1, 5, 2, 3, 15]

In [102]:
def EvaluasiDCG(GroundTruth, TopNRekomendasi, N):
    # Inisialisasi skor DCG (Discounted Cumulative Gain) ke 0.0
    dcg_score = 0.0
    for i in range(min(N, len(TopNRekomendasi))):
        # Jika item rekomendasi ada dalam GroundTruth
        if TopNRekomendasi[i] in GroundTruth:
            # rumus DCG
            dcg_score += 1 / np.log2(i + 2)  
            print(f"Item {TopNRekomendasi[i]} ada di GroundTruth pada posisi {i+1}, DCG: {dcg_score}")
    #return nilai DCG
    return dcg_score

In [103]:
cal_DCG = EvaluasiDCG(groundTruth, TopNrek, 4)
print(f"Nilai DCG: {cal_DCG}")

Item 1 ada di GroundTruth pada posisi 1, DCG: 1.0
Item 2 ada di GroundTruth pada posisi 3, DCG: 1.5
Item 3 ada di GroundTruth pada posisi 4, DCG: 1.9306765580733931
Nilai DCG: 1.9306765580733931


In [104]:
def EvaluasiIDCG(N):
    # Inisialisasi IDCG
    idcg = 0.0
    # Hitung IDCG untuk N item teratas
    for n in range(1, N + 1):
        # Menghitung nilai relevansi ideal
        relevansi_ideal = 1  # Asumsikan rating tertinggi adalah 1
        # Menambahkan ke IDCG dengan formula relevansi/log2 posisi
        idcg += relevansi_ideal / math.log2(n + 1)
        print(f"Item ideal pada posisi {n}, IDCG: {idcg}")
    # Mengembalikan nilai IDCG
    return idcg

In [105]:
cal_IDCG = EvaluasiIDCG(4)
print(f"Nilai IDCG: {cal_IDCG}")

Item ideal pada posisi 1, IDCG: 1.0
Item ideal pada posisi 2, IDCG: 1.6309297535714575
Item ideal pada posisi 3, IDCG: 2.1309297535714578
Item ideal pada posisi 4, IDCG: 2.5616063116448506
Nilai IDCG: 2.5616063116448506


In [106]:
def EvaluasiNDCG(GroundTruth, TopNRekomendasi, N):
    # Hitung DCG
    dcg = EvaluasiDCG(GroundTruth, TopNRekomendasi, N)
    print(f"Nilai DCG: {dcg}")
    # Hitung IDCG
    idcg = EvaluasiIDCG(N)
    print(f"Nilai IDCG: {idcg}")
    # Menghitung NDCG
    rumusNDCG = dcg / idcg if idcg != 0 else 0

    return rumusNDCG


In [107]:
calNDCG = EvaluasiNDCG(groundTruth, convertList1DFull, 3)
print(f"Nilai NDCG: {calNDCG}")

Nilai DCG: 0.0
Item ideal pada posisi 1, IDCG: 1.0
Item ideal pada posisi 2, IDCG: 1.6309297535714575
Item ideal pada posisi 3, IDCG: 2.1309297535714578
Nilai IDCG: 2.1309297535714578
Nilai NDCG: 0.0
